In [2]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()

In [3]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## Задание

- поделить на test и train (test не меньше чем 25%)
- поэкспериментировать с глубиной дерева, посмотреть на графике оценку на test и train
- подобрать наилучшее дерево с RandomizedSearch, визуализировать его (* с помощью GridSearch)
- проанализировать признаки (features), отсортировать их по важности, выбрать 2 наиболее важные
- построить confusion_matrix для наилучшего дерева